## Extração

In [3]:
# import de libs necessárias para o processo de data cleaning

import numpy as np
import pandas as pd
import os
from datetime import datetime
import time
import urllib.request
import json
import zipfile

start_time = time.time()

# Bibliotecas de banco devem vir por último
import psycopg2
from psycopg2 import sql
import sqlalchemy
from sqlalchemy import create_engine

In [4]:
# armazenando os ids de cada arquivo que serão baixados posteriormente
files_ids = ["99b42b09-95af-47de-8411-ab99c380c3ef",
             "9664de94-9f07-4adc-848d-b6ef56510762",
             "bceb5759-5500-49db-bc86-b038892acc06",
             "ca7fb968-3a2c-44ff-a2e8-730d1a689407",
              "c675899c-69d9-4dc8-bb11-00afc9636a3b"]

In [5]:
# Diretório onde o dataset será salvo
dataset_dir = "datasets"
os.makedirs(dataset_dir, exist_ok=True)

try:
    # URL direta do arquivo JSON
    for fileId in files_ids:
        url = f'http://dados.recife.pe.gov.br/datastore/dump/{fileId}?format=json'
        # Nome do arquivo de saída
        output_path = os.path.join(dataset_dir, f"{fileId}.json")
        try:
            print("Baixando arquivo JSON...")
            urllib.request.urlretrieve(url, output_path)
            print(f"Arquivo salvo em: {output_path}")
        except Exception as e:
            print(f"Erro ao baixar o arquivo: {e}")
    print("Todos os downloads foram concluídos com sucesso!")
except Exception as e:
    pass


Baixando arquivo JSON...
Arquivo salvo em: datasets\99b42b09-95af-47de-8411-ab99c380c3ef.json
Baixando arquivo JSON...
Erro ao baixar o arquivo: HTTP Error 404: Not Found
Baixando arquivo JSON...
Arquivo salvo em: datasets\bceb5759-5500-49db-bc86-b038892acc06.json
Baixando arquivo JSON...
Arquivo salvo em: datasets\ca7fb968-3a2c-44ff-a2e8-730d1a689407.json
Baixando arquivo JSON...
Arquivo salvo em: datasets\c675899c-69d9-4dc8-bb11-00afc9636a3b.json
Todos os downloads foram concluídos com sucesso!


In [ ]:

df = None
successful_files = 0

for fileName in files_ids:
    json_dir = f"datasets/{fileName}.json"
    # Abre e carrega o conteúdo do arquivo JSON em um dicionário Python
    if os.path.exists(json_dir):
        
        successful_files += 1

        with open(json_dir, 'r', encoding='utf-8') as f:
            json_data = json.load(f)
            
        # Extrai os nomes das colunas da lista "fields" e as tuplas da lista "records"
        tuples = json_data['records']
        cols = [field['id'] for field in json_data['fields']]
        
        # se for o primeiro json lido, vamos instanciar nosso dataFrame pandas que será usado no resto da aplicação
        if(fileName == files_ids[0]):
            df = pd.DataFrame(data=tuples, columns=cols)
        else:
            # caso contrário, vamos criar um dataframe temporário e concatenar com o já existente
            temp_df =  pd.DataFrame(data=tuples, columns=cols)
            df = pd.concat([df, temp_df], ignore_index=True)
        
print("Dataframe inicializado com sucesso!")
print(f"{successful_files} jsons adicionados ao df")

Dataframe inicializado com sucesso!
4 jsons adicionados ao df


In [10]:
# Vamos dar uma olhadinha no dataframe pra ver se tá tudo certo
print(f"{df.shape[0]} linhas e {df.shape[1]} colunas")
df.head()
#Verificando as colunas do dataframe
df.columns

73996 linhas e 14 colunas


Index(['_id', 'faixa_etaria', 'idade', 'sexo', 'raca_cor', 'municipio',
       'grupo', 'categoria', 'lote', 'vacina_fabricante', 'descricao_dose',
       'cnes', 'sistema_origem', 'data_vacinacao'],
      dtype='object')

In [11]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine, text
import pandas as pd

# carregar o arquivo JSON com as configurações de conexão
with open('../config.json', 'r') as config_file:
    config = json.load(config_file)


# 0. Criar banco covid_elt se não existir
try:
    conn_postgres = psycopg2.connect(
        dbname="postgres",
        user=config["user"],
        password=config["password"],
        host=config["host"],
        port=config["port"]
    )
    conn_postgres.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cur = conn_postgres.cursor()
    cur.execute("SELECT 1 FROM pg_database WHERE datname='covid_elt';")
    exists = cur.fetchone()
    if not exists:
        cur.execute("CREATE DATABASE covid_elt;")
        print("Banco 'covid_elt' criado com sucesso!")
    else:
        print("Banco 'covid_elt' já existe.")
    cur.close()
    conn_postgres.close()
except Exception as e:
    print(f"Erro ao criar banco: {e}")

# 1. Conexão manual com client_encoding forçado ao banco covid_elt
conn = psycopg2.connect(
    dbname="covid_elt",
    user=config["user"],
    password=config["password"],
    host=config["host"],
    port=config["port"]
)
conn.set_client_encoding('UTF8')  # forçando o encoding correto

# 2. Conectando com SQLAlchemy usando esse conn
engine = create_engine("postgresql+psycopg2://", creator=lambda: conn)

nome_tabela = "stg_casos_covid"

# 3. Preparação dos dados (só pra garantir segurança total)
df.fillna('', inplace=True)
df = df.astype(str)

# 4. Inserção em chunks
chunk_size = 1000
for start in range(0, len(df), chunk_size):
    end = min(start + chunk_size, len(df))
    df_chunk = df.iloc[start:end]
    try:
        df_chunk.to_sql(nome_tabela, con=engine, if_exists='append' if start > 0 else 'replace', index=False)
        print(f"Inseridos registros de {start} a {end - 1}")
    except Exception as e:
        print(f"Erro ao inserir registros de {start} a {end - 1}: {e}")
        break

# 5. Verificação
try:
    with engine.connect() as conn_check:
        result = conn_check.execute(text(f"SELECT * FROM {nome_tabela} LIMIT 1"))
        for row in result:
            print(row)
    print(f"Tabela '{nome_tabela}' carregada com sucesso.")
except Exception as e:
    print(f"Erro ao consultar a tabela: {e}")


Banco 'covid_elt' criado com sucesso!
Inseridos registros de 0 a 999
Inseridos registros de 1000 a 1999
Inseridos registros de 2000 a 2999
Inseridos registros de 3000 a 3999
Inseridos registros de 4000 a 4999
Inseridos registros de 5000 a 5999
Inseridos registros de 6000 a 6999
Inseridos registros de 7000 a 7999
Inseridos registros de 8000 a 8999
Inseridos registros de 9000 a 9999
Inseridos registros de 10000 a 10999
Inseridos registros de 11000 a 11999
Inseridos registros de 12000 a 12999
Inseridos registros de 13000 a 13999
Inseridos registros de 14000 a 14999
Inseridos registros de 15000 a 15999
Inseridos registros de 16000 a 16999
Inseridos registros de 17000 a 17999
Inseridos registros de 18000 a 18999
Inseridos registros de 19000 a 19999
Inseridos registros de 20000 a 20999
Inseridos registros de 21000 a 21999
Inseridos registros de 22000 a 22999
Inseridos registros de 23000 a 23999
Inseridos registros de 24000 a 24999
Inseridos registros de 25000 a 25999
Inseridos registros de 2